<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="./images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by Özlem Salehi</td>    
</table>
<hr>

# QUBO for Music Generation

In this notebook, we will model the music generation process by quadratic unconstrained binary optimization.

The codes in this notebook are used to generate the music pieces presented in the paper ["Music Composition Using Quantum Annealing"](https://arxiv.org/abs/2201.10557). You can read the paper to get a deeper insight. The content is also published as a chapter in the following reference:

Arya, A., Botelho, L., Cañete, F., Kapadia, D., & Salehi, Ö. (2022). Applications of Quantum Annealing to Music Theory. In Quantum Computer Music: Foundations, Methods and Advanced Concepts (pp. 373-406). Cham: Springer International Publishing.

## Preliminaries

### Imports

You need to import the following to run the codes. If you have made a successfull installation, you should be able to run the following cell without any error. 

In [37]:
# D-Wave modules
import dimod
import neal
from dwave.system.samplers.leap_hybrid_sampler import LeapHybridSampler
from dwave.system import EmbeddingComposite, DWaveSampler
import dwave_networkx as dnx 

# Pyqubo will be used formulate the models
from pyqubo import Binary, Constraint, LogEncInteger

# pickle will be used for saving data
import pickle

# This is a built in module which we will use in the header of the for loops.
from itertools import product

### Helper functions 

We will list some functions that will be used in the rest of the notebook.

<b>Storing results</b>

The sampleset returned by D-Wave is converted in to a serializable object which is convenient for storing and pickle is used to store it.

In [38]:
def store_results(filename, sampleset):
    """Stores the sampleset obtained from D-Wave
    
    :param filename: name of the file 
    :type filename: string
    :param sampleset: sampleset to be stored
    :type sampleset: dimod.SampleSet
    """
    sdf = sampleset.to_serializable()
    with open(filename, 'wb') as handle:
        pickle.dump(sdf, handle)

<b> Annealing experiment</b>

This function is used to perform the annealing experiment. Annealing type is either 's', 'h' or 'q', representing simulated, hybrid and quantum respectively. The parameter `ns` which stands for number of sweeps is needed only for simulated annealing. `at` is the annealing time for quantum annealing and `cs` determines the chain strength. For hybrid annealing, no parameters are used. If any of the parameters are not provided, then their default values are used.

In [39]:
def anneal(bqm, anneal_type, ns=1000, nr=1000, at=20, cs=1):
    """Runs an annealing experiment and returns the sampleset
    
    :param bqm: binary quadratic model to be sampled
    :type bqm: BinaryQuadraticModel
    :param type: type of the annealing (simulated, hybrid, quantum)
    :type: string
    :param ns: Number of sweeps
    :type ns: int
    :param nr: Number of samples
    :type nr: int
    :param at: Annealing time
    :type at: int
    :param cs: Chain strength 
    :type cs: float
    :return: sampleset
    :rtype: dimod.SampleSet
    """
    if anneal_type == 's':
        s = neal.SimulatedAnnealingSampler()
        sampleset = s.sample(bqm, beta_range=(5, 100), num_sweeps=ns, num_reads=ns,
                             beta_schedule_type='geometric')
    elif anneal_type == 'h':
        s = LeapHybridSampler()
        sampleset = s.sample(bqm)
    elif anneal_type == 'q':
        s = EmbeddingComposite(DWaveSampler())
        sampleset = s.sample(bqm, chain_strength = cs, annealing_time = at, num_reads=1000)

    results, energies = [], []

    for datum in sampleset.data(fields=["sample", "energy"]):
        energies.append(datum.energy)
        results.append(datum[0])
    return sampleset

<b> Converting sample into a pitch sequence </b>

The output we get from D-Wave is not a pitch sequence but a list of binary variables and their values. After getting this output, we need to convert it into a pitch sequence.

In [40]:
def translate_sample(sample, P, n):
    """Converts a given sample into a pitch sequence
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param P: List of the pitches
    :param type P: list
    :param n: Number of notes in the sequence
    :param type n: int
    :return: sequence of pitches
    :rtype: list
    """
    seq = []
    for i,p in product(range(1,n+1),P):
        if sample[f"x_{i}_{p}"]==1:
            seq.append(p)
    return seq

<b> Converting sample into a chord sequence

In case we would like to create a chord sequence, there will be three notes at each time point. The following function creates a chord sequence from the sample.

In [41]:
from itertools import product

def translate_chord(sample,P,n):
    """Converts a given sample into a chord sequence
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param P: List of the pitches
    :param type P: list
    :param n: Number of notes in the sequence
    :param type n: int
    :return: sequence of chords
    :rtype: list
    """
    seq = {i:[] for i in range(1,n+1)}
    for i,p in product(range(1,n+1),P):
        if sample[f"x_{i}_{p}"]==1:
            seq[i].append(p)
    return seq

<b> Checking if constraints are broken using pyqubo

pyqubo has a built-in method to check whether the constraints defined using the `Constraint` keyword are broken or not. (It does not detect if there are penalty values resulting from the terms directly added to the objective.)

In [42]:
def check_constraints(sample, model):    
    """Prints the violated constraints in a given sample
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param model: Model that is representing the qubo problem
    :param type P: Pyqubo model
    """
    dec = model.decode_sample(sample, vartype='BINARY')
    print(dec.constraints(only_broken=True))

----

### Example 1 (Page 13) - Basic model for melody generation

<b> Creating the model

Suppose that we want to generate a melody consisting of $n$ notes, where the pitches belong to the set $P=\{p_1,p_2,\dots,p_k \}$. We will define the binary variables $x_{i,j}$ for $i \in [n]$ and $j \in P$.
\begin{equation}\label{eq:model}
x_{i,j} =  \begin{cases}%
1      & \text{note at position $i$ is $j$,}\\
0 & \text{otherwise.}
\end{cases}
\end{equation}
The total number of variables required in this formulation is $n|P|$. Note that we aim to generate $n$ notes simultaneously. At the end of the optimization process, we will obtain an assignment to the binary variables $x_{i,j}$, which will indicate the pitch of the note selected for each position. 

#### Constraints

Next, we will define some constraints which can be included in the objective function using the penalty method. 

The first rule we need to incorporate is that only one of the variables $x_{i,j}$ is equal to 1 for each position $i$. The rule is necessary as exactly one pitch should be selected for each time point. 

This is equivalent to having the following constraint for each $i \in [n]$.
\begin{equation}\label{eq:single_note}
    \sum_{j \in P} x_{i,j} = 1
\end{equation}

#### Code

We first create a list of pitches `P` and set `n`, which is the number of notes in the music piece to be generated. 

In [43]:
P = ['C4', 'D4', 'E4','G4' ]
n = 5

Next we add the single note constraint, which is defined as 
$
\left ( 1- \sum_{j \in P} x_{i,j} \right )^2
$
for each $i=1,\dots,5$.

We use pyqubo to formulate the model. 

- $H$ will represent our QUBO formulation.
- Note that we have a for loop going through each $i$ and we add the constraint corresponding to $i$ to $H$.
- `Binary(x)` keyword defines a binary variable with name `x`
-  `sum(Binary(f"x_{i}_{j}") for j in P)` represents the sum $\sum_{j \in P} x_{i,j}$
- `Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")` adds the constraint $
\left ( 1- \sum_{j \in P} x_{i,j} \right )^2
$ to the model with the name `Single_note_{i}`


In [44]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{j}") for j in P))**2, f"Single_note_{i}")

We can print `H` to see what it looks like.

In [45]:
H

(Constraint(label=Single_note_5,((Binary(x_5_G4)+Binary(x_5_E4)+Binary(x_5_C4)+Binary(x_5_D4))*Num(-1.000000)+Num(1.000000))*((Binary(x_5_G4)+Binary(x_5_E4)+Binary(x_5_C4)+Binary(x_5_D4))*Num(-1.000000)+Num(1.000000)))+Constraint(label=Single_note_4,((Binary(x_4_G4)+Binary(x_4_E4)+Binary(x_4_C4)+Binary(x_4_D4))*Num(-1.000000)+Num(1.000000))*((Binary(x_4_G4)+Binary(x_4_E4)+Binary(x_4_C4)+Binary(x_4_D4))*Num(-1.000000)+Num(1.000000)))+Constraint(label=Single_note_3,((Binary(x_3_G4)+Binary(x_3_E4)+Binary(x_3_C4)+Binary(x_3_D4))*Num(-1.000000)+Num(1.000000))*((Binary(x_3_G4)+Binary(x_3_E4)+Binary(x_3_C4)+Binary(x_3_D4))*Num(-1.000000)+Num(1.000000)))+Constraint(label=Single_note_1,((Binary(x_1_G4)+Binary(x_1_E4)+Binary(x_1_C4)+Binary(x_1_D4))*Num(-1.000000)+Num(1.000000))*((Binary(x_1_G4)+Binary(x_1_E4)+Binary(x_1_C4)+Binary(x_1_D4))*Num(-1.000000)+Num(1.000000)))+Constraint(label=Single_note_2,((Binary(x_2_G4)+Binary(x_2_E4)+Binary(x_2_C4)+Binary(x_2_D4))*Num(-1.000000)+Num(1.000000))*((B

<b> Running the experiment

To run an annealing experiment using Ocean SDK, we need a BinaryQuadraticModel.

First we use pyqubo's `compile` function to generate the `model` object, which is then converted into a QUBO using `to_qubo()` function of pyqubo. 

From `qubo` and `of` representing the QUBO model and the offset respectively, we create a BinaryQuadraticModel object of D-Wave.

In [46]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

Let's print `bqm` as well.

In [47]:
bqm

BinaryQuadraticModel({'x_2_G4': -1.0, 'x_2_C4': -1.0, 'x_5_G4': -1.0, 'x_5_E4': -1.0, 'x_5_C4': -1.0, 'x_3_C4': -1.0, 'x_3_D4': -1.0, 'x_2_E4': -1.0, 'x_1_E4': -1.0, 'x_1_D4': -1.0, 'x_4_E4': -1.0, 'x_4_C4': -1.0, 'x_5_D4': -1.0, 'x_1_C4': -1.0, 'x_1_G4': -1.0, 'x_4_D4': -1.0, 'x_2_D4': -1.0, 'x_3_G4': -1.0, 'x_4_G4': -1.0, 'x_3_E4': -1.0}, {('x_2_C4', 'x_2_G4'): 2.0, ('x_5_E4', 'x_5_G4'): 2.0, ('x_5_C4', 'x_5_G4'): 2.0, ('x_5_C4', 'x_5_E4'): 2.0, ('x_3_D4', 'x_3_C4'): 2.0, ('x_2_E4', 'x_2_G4'): 2.0, ('x_2_E4', 'x_2_C4'): 2.0, ('x_1_D4', 'x_1_E4'): 2.0, ('x_4_C4', 'x_4_E4'): 2.0, ('x_5_D4', 'x_5_G4'): 2.0, ('x_5_D4', 'x_5_E4'): 2.0, ('x_5_D4', 'x_5_C4'): 2.0, ('x_1_C4', 'x_1_E4'): 2.0, ('x_1_C4', 'x_1_D4'): 2.0, ('x_1_G4', 'x_1_E4'): 2.0, ('x_1_G4', 'x_1_D4'): 2.0, ('x_1_G4', 'x_1_C4'): 2.0, ('x_4_D4', 'x_4_E4'): 2.0, ('x_4_D4', 'x_4_C4'): 2.0, ('x_2_D4', 'x_2_G4'): 2.0, ('x_2_D4', 'x_2_C4'): 2.0, ('x_2_D4', 'x_2_E4'): 2.0, ('x_3_G4', 'x_3_C4'): 2.0, ('x_3_G4', 'x_3_D4'): 2.0, ('x_4_G4

The next step is to run the experiment. We use the `anneal` function we have previously written. In this example, we select quantum annealing by providing the parameter `q`. As we don't provide any additional parameters, default parameters will be used for annealing time and chain strength.

<i>Note: If you did not configure D-Wave access or you can not configure it due to your country of residence, then you can use 's' keyword to use simulated annealer instead. 

In [48]:
sampleset = anneal(bqm, 'q')

<b> Interpreting the results

Let's see what samples look like. 

In [49]:
sampleset

SampleSet(rec.array([([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], 0., 3, 0.),
           ([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], 0., 1, 0.),
           ([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 0., 2, 0.),
           ([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], 0., 2, 0.),
           ([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 0., 1, 0.),
           ([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], 0., 1, 0.),
           ([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 0., 1, 0.),
           ([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0], 0., 1, 0.),
           ([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0], 0., 2, 0.),
           ([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0], 0., 2, 0.),
           ([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0], 0., 1, 0.),
           ([1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0

When you consider a line from the samples list, the list is the value of the binary variables, 0 indicates that the energy is 0, and 1 indicates that this specific sample is observed only once.
`[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], 0., 1)`

The order of the variables can be seen from the list of variables.

`Variables(['x_1_C4', 'x_1_D4', 'x_1_E4', 'x_1_G4', 'x_2_C4', 'x_2_D4', 'x_2_E4', 'x_2_G4', 'x_3_C4', 'x_3_D4', 'x_3_E4', 'x_3_G4', 'x_4_C4', 'x_4_D4', 'x_4_E4', 'x_4_G4', 'x_5_C4', 'x_5_D4', 'x_5_E4', 'x_5_G4']`

Finally, let's print the corresponding note sequences to the sampleset we have obtained. We take the first 5 samples.

In [50]:
for sample in sampleset.samples()[:5]:
    seq = translate_sample(sample,P,n)
    print(seq)

['D4', 'D4', 'D4', 'D4', 'D4']
['D4', 'E4', 'C4', 'E4', 'D4']
['D4', 'C4', 'E4', 'D4', 'C4']
['E4', 'E4', 'D4', 'C4', 'C4']
['G4', 'C4', 'C4', 'G4', 'C4']


We can also print all samples and their energies.

In [51]:
for data in sampleset.data():
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)

['D4', 'D4', 'D4', 'D4', 'D4'] 0.0
['D4', 'E4', 'C4', 'E4', 'D4'] 0.0
['D4', 'C4', 'E4', 'D4', 'C4'] 0.0
['E4', 'E4', 'D4', 'C4', 'C4'] 0.0
['G4', 'C4', 'C4', 'G4', 'C4'] 0.0
['E4', 'E4', 'D4', 'C4', 'G4'] 0.0
['E4', 'E4', 'D4', 'C4', 'D4'] 0.0
['E4', 'E4', 'G4', 'D4', 'C4'] 0.0
['C4', 'D4', 'C4', 'G4', 'C4'] 0.0
['E4', 'D4', 'C4', 'D4', 'E4'] 0.0
['C4', 'G4', 'D4', 'G4', 'C4'] 0.0
['D4', 'G4', 'E4', 'C4', 'C4'] 0.0
['C4', 'G4', 'C4', 'E4', 'C4'] 0.0
['G4', 'D4', 'C4', 'C4', 'D4'] 0.0
['D4', 'C4', 'G4', 'G4', 'D4'] 0.0
['C4', 'E4', 'D4', 'D4', 'D4'] 0.0
['D4', 'C4', 'G4', 'D4', 'D4'] 0.0
['D4', 'E4', 'G4', 'G4', 'D4'] 0.0
['D4', 'G4', 'E4', 'D4', 'C4'] 0.0
['C4', 'G4', 'G4', 'G4', 'C4'] 0.0
['C4', 'C4', 'C4', 'E4', 'D4'] 0.0
['G4', 'G4', 'E4', 'G4', 'G4'] 0.0
['E4', 'G4', 'C4', 'G4', 'D4'] 0.0
['D4', 'E4', 'D4', 'G4', 'E4'] 0.0
['G4', 'G4', 'E4', 'D4', 'G4'] 0.0
['G4', 'E4', 'D4', 'G4', 'G4'] 0.0
['E4', 'G4', 'D4', 'G4', 'G4'] 0.0
['C4', 'E4', 'C4', 'D4', 'G4'] 0.0
['E4', 'C4', 'C4', '

['G4', 'D4', 'E4', 'G4', 'E4'] 0.0
['E4', 'D4', 'G4', 'E4', 'C4'] 0.0
['C4', 'D4', 'D4', 'E4', 'C4'] 0.0
['D4', 'G4', 'C4', 'E4', 'E4'] 0.0
['C4', 'G4', 'G4', 'E4', 'G4'] 0.0
['D4', 'D4', 'D4', 'D4', 'E4'] 0.0
['C4', 'C4', 'E4', 'E4', 'G4'] 0.0
['C4', 'G4', 'E4', 'G4', 'D4'] 0.0
['C4', 'G4', 'D4', 'C4', 'C4'] 0.0
['D4', 'E4', 'G4', 'G4', 'E4'] 0.0
['D4', 'C4', 'C4', 'C4', 'D4'] 0.0
['G4', 'C4', 'D4', 'G4', 'G4'] 0.0
['E4', 'G4', 'D4', 'E4', 'E4'] 0.0
['G4', 'G4', 'G4', 'D4', 'C4'] 0.0
['D4', 'E4', 'C4', 'G4', 'D4'] 0.0
['E4', 'E4', 'C4', 'D4', 'G4'] 0.0
['E4', 'E4', 'C4', 'C4', 'E4'] 0.0
['D4', 'E4', 'G4', 'E4', 'C4'] 0.0
['G4', 'C4', 'C4', 'C4', 'G4'] 0.0
['G4', 'C4', 'G4', 'D4', 'C4'] 0.0
['G4', 'D4', 'C4', 'C4', 'E4'] 0.0
['C4', 'E4', 'G4', 'C4', 'D4'] 0.0
['E4', 'C4', 'C4', 'E4', 'D4'] 0.0
['G4', 'E4', 'G4', 'G4', 'E4'] 0.0
['G4', 'C4', 'G4', 'E4', 'D4'] 0.0
['E4', 'E4', 'E4', 'G4', 'C4'] 0.0
['D4', 'G4', 'C4', 'G4', 'C4'] 0.0
['C4', 'D4', 'C4', 'C4', 'C4'] 0.0
['C4', 'G4', 'D4', '

['G4', 'D4', 'D4', 'G4', 'D4'] 0.0
['C4', 'D4', 'G4', 'D4', 'C4'] 0.0
['G4', 'G4', 'D4', 'D4', 'D4'] 0.0
['E4', 'C4', 'E4', 'G4', 'C4'] 0.0
['G4', 'E4', 'G4', 'G4', 'C4'] 0.0
['E4', 'C4', 'C4', 'D4', 'D4'] 0.0
['G4', 'C4', 'G4', 'C4', 'D4'] 0.0
['C4', 'C4', 'E4', 'G4', 'C4'] 0.0
['G4', 'D4', 'E4', 'E4', 'C4'] 0.0
['C4', 'E4', 'G4', 'G4', 'E4'] 0.0
['E4', 'E4', 'G4', 'G4', 'E4'] 0.0
['G4', 'G4', 'D4', 'C4', 'D4'] 0.0
['G4', 'D4', 'C4', 'D4', 'G4'] 0.0
['G4', 'D4', 'G4', 'E4', 'G4'] 0.0
['G4', 'C4', 'D4', 'D4', 'G4'] 0.0
['E4', 'D4', 'G4', 'E4', 'E4'] 0.0
['C4', 'E4', 'D4', 'C4', 'E4'] 0.0
['D4', 'C4', 'E4', 'C4', 'E4'] 0.0
['E4', 'D4', 'G4', 'D4', 'G4'] 0.0
['C4', 'G4', 'D4', 'G4', 'E4'] 0.0
['D4', 'E4', 'E4', 'G4', 'G4'] 0.0
['G4', 'E4', 'E4', 'G4', 'D4'] 0.0
['C4', 'C4', 'D4', 'D4', 'G4'] 0.0
['E4', 'G4', 'G4', 'G4', 'G4'] 0.0
['E4', 'E4', 'G4', 'D4', 'G4'] 0.0
['D4', 'G4', 'D4', 'E4', 'G4'] 0.0
['E4', 'G4', 'D4', 'G4', 'C4'] 0.0
['E4', 'D4', 'G4', 'G4', 'E4'] 0.0
['E4', 'E4', 'G4', '

You can get the sample with the lowest energy using `sampleset.first.sample`. Since there are multiple of them in this case, this will return you the first sample in the list.

In [52]:
sampleset.first.sample

{'x_1_C4': 0,
 'x_1_D4': 1,
 'x_1_E4': 0,
 'x_1_G4': 0,
 'x_2_C4': 0,
 'x_2_D4': 1,
 'x_2_E4': 0,
 'x_2_G4': 0,
 'x_3_C4': 0,
 'x_3_D4': 1,
 'x_3_E4': 0,
 'x_3_G4': 0,
 'x_4_C4': 0,
 'x_4_D4': 1,
 'x_4_E4': 0,
 'x_4_G4': 0,
 'x_5_C4': 0,
 'x_5_D4': 1,
 'x_5_E4': 0,
 'x_5_G4': 0}

As the lowest energy sample has energy 0, we expect that no constraint is violated. We can check it also using pyqubo. The returned dictionary will contain the list of violated constraints, or {} in case no constraint is violated. 

In [53]:
check_constraints(sampleset.first.sample, model)  

{}


If you want to store the results, you can run the following cell. 

In [54]:
filename ="experiment1"
store_results(filename, sampleset)

---

### Example 2 (page 14) - Avoiding certain consecutive notes

Suppose that we want to add a restriction that the note $p_l$ does not appear after the note $p_k$. This is useful for avoiding particular intervals and amending our model. 

Such a restriction can be incorporated into the QUBO formulation by adding the following term to the objective function multiplied with a suitable penalty coefficient $C$ for each $i \in [n-1]$.
\begin{equation}\label{eq:consec} 
x_{i,p_k}x_{i+1,p_l}     
\end{equation}

Note that when both variables equal 1 simultaneously, a penalty of $C$ is added to the objective function. Alternatively, we can express the same rule using the following constraint.
\begin{equation}\label{eq:constraint}
    x_{i,p_k} + x_{i+1,p_l} \leq 1     
\end{equation}

In this case, the inequality should be first transformed into equality by using slack variables and then added to the objective function.

<b> Creating the model

We use the same `P` and `n` as before.

In [55]:
P = ['C4', 'D4', 'E4','G4' ]
n = 5

First we define the single note constraint.

In [56]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We want to enforce that G4 does not follow D4. We add the term ` Binary(f"x_{i}_D4")*Binary(f"x_{i+1}_G4")` directly to the objective for each $i$. Note that the penalty coefficient is 1 in this case.

In [57]:
for i in range(1,n+1):
    H += Binary(f"x_{i}_D4")*Binary(f"x_{i+1}_G4")     

Now let us consider a rule saying that the same note does not appear more than twice in a row. Similar to what we had above, the term $x_{i,p_j}x_{i+1,p_j}x_{i+2,p_j} $ can be added to the objective for each $i\in[n-2]$ and $j \in P$. However, this is not a quadratic term, and quadratization is needed to obtain a QUBO. 

Alternatively, the rule can be expressed by the following constraint, which forces that at most two of the variables are equal to 1 simultaneously. 

\begin{equation}\label{eq:samenote}
 x_{i,j} + x_{i+1,j} + x_{i+2,j} \leq 2
\end{equation}

As we have an inequality, we need to introduce a slack variable $s_{i,j}$. 

$$ x_{i,j} + x_{i+1,j} + x_{i+2,j} +s_{i,j} = 2$$

The upper bound for the slack variable is 2, as all three binary variables can be equal to 0 simulatenously. Hence we need an integer variable $s$. We use `LogEncInteger(f"s_{i}_{j}",(0,2))` from pyqubo to create an integer variable named `s_i_j`, with the lower and upper bounds (0,2). 

Finally, we convert it to the following constraint add to our model.

$ (x_{i,j} + x_{i+1,j} + x_{i+2,j} +s_{i,j} - 2)^2$

Notice that the penalty value is set as 1.

In [58]:
for i in range(1,n-1):
    for j in P:
        H += Constraint((Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

<b> Running the experiment

In [59]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We increase the chain strength to 1.5 this time.

In [60]:
sampleset = anneal(bqm, 's', cs=1.5)

<b>Interpreting the results

Let's print the first 5 samples and their energies.

In [61]:
for i, data in enumerate(sampleset.data()):
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)
    if i==5:
        break

['G4', 'D4', 'E4', 'G4', 'C4'] 0.0
['G4', 'E4', 'G4', 'E4', 'G4'] 0.0
['D4', 'E4', 'C4', 'D4', 'E4'] 0.0
['C4', 'E4', 'G4', 'C4', 'D4'] 0.0
['D4', 'E4', 'E4', 'C4', 'C4'] 0.0
['E4', 'G4', 'C4', 'E4', 'E4'] 0.0


In [62]:
filename = "experiment2"
store_results(filename, sampleset)

---

### Example 4 (Page 15) - Diatonic scale and tendency notes

All music pieces have a definite key signature. Therefore it is often more suitable to work with pitches from a particular scale.

For simplicity, let us consider the 8 degrees of a diatonic scale. In this case, the set $P$ consists of $d_1,d_2,\dots,d_8$ where $d_j$ is the $j$'th degree of the scale. Instead of defining the variables by the pitches, we will define them through degrees for each $i\in[n]$ and $d_j\in P$.

\begin{equation}\label{eq:modeldiatonic}
x_{i,j} =  \begin{cases}%
1      & \text{note at position $i$ is $d_j$,}\\
0 & \text{otherwise.}
\end{cases}
\end{equation}

As a result of the optimization procedure, we will obtain a degree sequence, which can then be translated into a note sequence based on the chosen scale. Hence, our model is readily adaptable to different scales. Furthermore, the rules described in the previous subsections are still applicable. 

Some notes in the scale are less stable than the others which are known as the \textit{tendency notes} and tend to resolve to the stable ones. Let us examine how to incorporate rules about tendency notes.  According to the rule, degrees 2, 4, and 6 resolve down by one step, and degree 7 resolves to the octave. To reflect the rule about tendency notes, for each $i \in [n-1]$, we will add the terms defined in Eq.~\ref{eq:tendency} to the objective.

\begin{equation}\label{eq:tendency}
x_{i,2}(1-x_{i+1,1}),~~
x_{i,4}(1-x_{i+1,3}),~~
x_{i,6}(1-x_{i+1,5}),~~
x_{i,7}(1-x_{i+1,8})
\end{equation}

<b>Creating the model

This time, we take the diatonic scale and select `P` as follows. We will generate 20 notes.

In [75]:
P = [1,2,3,4,5,6,7,8]
n = 20

We start by adding the single note constraint.

In [76]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We initialize the first and the last note of the sequence.

In [77]:
H += (1-Binary(f"x_1_1")) + (1-Binary(f"x_20_1"))

Next we add the rules about tendency notes, by adding the following terms directly to the objective.

In [78]:
for i in range(1,n):
    H += Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))
    H += Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))
    H += Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))
    H += Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))

<b> Running the experiment

In [79]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

In [80]:
cs = 1.5
sampleset = anneal(bqm, 'q', cs)

<b> Interpreting the results

Let's print the first five samples and their energies.

In [81]:
for i,data in enumerate(sampleset.data()):
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)
    if i==6:
        break

[1, 8, 3, 2, 1, 5, 7, 8, 2, 1, 1, 5, 8, 7, 1, 6, 5, 8, 1, 5] 2.0
[1, 2, 1, 6, 5, 8, 5, 2, 1, 3, 3, 8, 8, 5, 7, 8, 2, 7, 1, 2, 1] 2.0
[1, 8, 1, 1, 1, 4, 3, 1, 3, 5, 8, 5, 3, 5, 1, 4, 3, 2, 7, 1, 3, 1] 3.0
[1, 2, 1, 7, 8, 3, 4, 3, 1, 3, 7, 3, 3, 5, 7, 1, 3, 7, 8, 5, 1] 3.0
[1, 8, 1, 3, 8, 7, 1, 8, 5, 7, 1, 8, 5, 3, 7, 8, 3, 3, 8, 3, 1] 3.0
[1, 8, 3, 8, 1, 3, 1, 8, 3, 7, 8, 1, 7, 8, 8, 1, 2, 7, 3, 5, 1] 3.0
[1, 8, 1, 1, 8, 5, 7, 2, 1, 1, 7, 8, 5, 1, 1, 3, 5, 7, 5, 1, 1] 3.0


In [82]:
filename = "experiment4"
store_results(filename, sampleset)

---

### Example 5 (Page 16-17) - Objective function

In order to differentiate between the feasible solutions, we can give some `rewards' to a particular sequence of notes, i.e. decrease their energy. For instance,  we might give a higher reward for pitch $\mathtt{D4}$ following $\mathtt{C4}$ vs. pitch $\mathtt{E4}$ following $\mathtt{C4}$. The way to accomplish this is to have the following term in the objective function.
\begin{equation}\label{eq:objective}
    -\sum_{ \substack{i\in [n-1]  \\ j,j' \in P}} W_{j,j'} x_{i,j} x_{i+1,j'}
\end{equation}
Here, $W_{j,j'}$ is the weight associated with having note $j'$ after note $j$. The larger the weight, the higher the reward we have in the objective function. Note that we have a negative sign in front of the equation, as we have a minimization problem and want to decrease the energy. The weights can be determined by analyzing some music pieces and forming a transition matrix of weights examining the consecutive notes. Below, we illustrate this idea through a simple music piece.


<b>Creating the model

We use the diatonic scale as the list of pitches and we will generate 20 notes.

In [83]:
P = [1,2,3,4,5,6,7,8]
n = 20

We define some penalty values. 

In [84]:
P1,P2,P3,P4,P5 = 5,3,1,1,1

We define the matrix of the weights.

In [85]:
M = [[0 for i in range(9)] for j in range(9)]
M[4][4]=2
M[4][3]=2
M[4][5]=1

M[5][6]=1
M[5][4]=1

M[6][6]=1
M[6][5]=1


M[2][1]=1
M[2][3]=1
M[2][2]=1

M[1][1]=1
M[1][2]=1

As always, we start by single note constraint. (Eq.18)

In [86]:
H = 0
for i in range(1,n+1):
    H += P1*Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We add the rule about the first and the last note. (Eq.26)

In [87]:
H += P2*Constraint(1-Binary(f"x_1_1"),"first") + P2*Constraint(1-Binary(f"x_20_1"),"last")

We include the rule about no more than two same note in a sequence. (Eq.22)

In [88]:
for i in range(1,n-1):
    for j in P:
        H += Constraint(P3*(Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

Next we add the rules about tendency notes (Eq.28), by adding the following terms directly to the objective.

In [89]:
for i in range(1,n):
    H += P4*Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))
    H += P4*Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))
    H += P4*Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))
    H += P4*Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))

Finally, we add the 'rewards' to our objective function, defined as $
    -\sum_{ \substack{i\in [n-1]  \\ j,j' \in P}} W_{j,j'} x_{i,j} x_{i+1,j'}
$. The weights we use are stored inside `M`. (Eq. 29)



In [90]:
for i in range(1,n+1):
    for j, jp in product(P,P):
        if M[j][jp]!=0:
            H += -P5*M[j][jp]*Binary(f"x_{i}_{j}")*Binary(f"x_{i+1}_{jp}")           

<b> Running the experiment

In [91]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We'll again use the hybrid solver.

In [92]:
sampleset = anneal(bqm, 'h')

<b> Interpreting the results

In [93]:
for data in sampleset.data():
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)

[1, 2, 1, 4, 3, 4, 3, 5, 4, 3, 4, 3, 4, 3, 1, 2, 1, 4, 3, 1] -19.0


In [94]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

{}


In [95]:
filename = "experiment5"
store_results(filename, sampleset)

---

### Example 6 (Page 17-18) - Rhythm generation

When generating a music piece, one option would be to generate the pitch sequence and the rhythmic sequence separately. In such a case, the idea will be similar to what we had previously. The set $S$ will consist of possible durations for the notes, such as whole, half, quarter, etc. The binary variables $y_{i,j}$ for $i \in [n]$ and $j \in D$ will be defined as follows.
\begin{equation}\label{eq:modelrhythm}
y_{i,j} =  \begin{cases}%
1      & \text{note at position $i$ has duration $j$,}\\
0 & \text{otherwise.}
\end{cases}
\end{equation}
 

Similarly, the first rule we need to incorporate is that only one of the variables $y_{i,j}$ is equal to 1 for each position $i$, which is expressed using the following constraint for each $i \in [n]$.
\begin{equation}\label{eq:singlerhythm}
    \sum_{j \in D} y_{i,j} = 1
\end{equation}

For the objective function, the same method can be used. Let us denote half note by $\mathtt{H}$, quarter note by $\mathtt{Q}$, dotted quarter note by $\mathtt{DQ}$ and eighth note by $\mathtt{E}$. 

<b> Creating the model

We identify quarter, dotted quarted, eighth and half notes by 0,1,2,3 respectively. Later on, we will use this `Pdict` to translate our sample back.

In [96]:
Pdict = {0:"Q", 1:"DQ", 2:"E", 3:"H"}
P = [0,1,2,3]
n = 20

We define the matrix of weights for the note durations.

In [97]:
M = [[0 for i in range(4)] for j in range(4)]
M[0][0]=11
M[0][1]=1

M[1][2] = 1
M[2][3]=1

Penalty values.

In [98]:
P1, P2, P3 = 50, 50, 1

We start by single note constraint. (Eq.18)

In [99]:
H = 0
for i in range(1,n+1):
    H += P1*Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We add the rule forcing that each duration appears at least twice. (Eq.36)

In [100]:
for d in P:
    H += P2*Constraint((2-sum(Binary(f"x_{i}_{d}") for i in range(1,n+1))+LogEncInteger(f"s_{d}",(0,14)))**2, f"At_least_two_{d}")

We add the 'rewards' reflecting the weights for consecutive durations. (Eq. 29)

In [101]:
for i in range(1,n+1):
    for j, jp in product(P,P):
        if M[j][jp]!=0:
            H += P3 * -M[j][jp]*Binary(f"x_{i}_{j}")*Binary(f"x_{i+1}_{jp}")

<b> Running the experiment

In [102]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We will use the hybrid solver

In [103]:
sampleset = anneal(bqm, 'h')

<b> Interpreting the results

In [104]:
for data in sampleset.data():
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)

[1, 2, 0, 0, 0, 3, 2, 3, 2, 2, 0, 2, 3, 2, 2, 0, 1, 0, 0, 0] -60.0


We can get the list of durations back using the dictionary.

In [105]:
L = [Pdict[s] for s in seq]
print(L)

['DQ', 'E', 'Q', 'Q', 'Q', 'H', 'E', 'H', 'E', 'E', 'Q', 'E', 'H', 'E', 'E', 'Q', 'DQ', 'Q', 'Q', 'Q']


In [106]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

{}


In [107]:
filename = f"experiment6"
store_results(filename, sampleset)

---

### Example 8 (Page 22) - Harmonizing melody

Suppose that we would like to add three-note chords, triads, to a given music piece. The notes appearing in the triads will be restricted to the set of pitches $P=\{1,2,3,4,5,6,7,8\}$, where the elements represent the degrees of the scale. Our binary variables $x_{i,j}$ for $i \in [n]$ and $j \in P$ are defined as follow.
\begin{equation}\label{eq:model_chords}
x_{i,j} =  \begin{cases}%
1      & \text{chord at position $i$ contains note $j$,}\\
0 & \text{otherwise.}
\end{cases}
\end{equation}

At each time point, we would like to have three notes. Hence, we are going to add the following constraint. 
\begin{equation}\label{eq:three_notes}
    \sum_{j \in P} x_{i,j} = 3
\end{equation}

Recall that we had already defined a constraint so that the melody begins and ends with the first degree of the scale. Similarly, we can enforce the first and the last triads to be built upon the first degree of the scale.  We can add the following terms to the objective function:

\begin{equation}\label{eq:first_last_chord}
    (1-x_{1,1}),~~(1-x_{1,3}),~~(1-x_{1,5}),~~(1-x_{n,1}),~~(1-x_{n,3}),~~(1-x_{n,5}).
\end{equation}

For each time point, one of the pitches in the triad should be the pitch of the melody. Let $N=[p_{1_j},p_{2_j},\dots,p_{n_j} ]$ be the note sequence that we would like to harmonize. We assume that each $p_{i_j} \in P $. To enforce that one of the three notes in the triad at time point $i$ is $p_{i_j}$, we add the term defined in Eq.~\ref{eq:note_i} to the objective function for all $i= 2,\dots,n-1$. (We are assuming that the first and the last pitch of the note sequence is the first degree of the scale.)

\begin{equation}\label{eq:note_i}
     (1-x_{i,p_{i_j}})
\end{equation}

We want to generate triads either obtained by taking the degrees of the scale as their root or their inversions. We want to avoid consecutive degrees from the scale and the degrees 1-7, 2-8, 1-8 to appear in the same triad by adding the following terms to the objective function 
for all $i= 2,\dots,n-1$, $|j-j'| \in \{1,6,7\}$.
\begin{equation}\label{eq:triad}
    x_{i,j}x_{i,j'}
\end{equation}

<b> Creating the model

The list `P` represents the degrees of the scale

In [119]:
P = [1,2,3,4,5,6,7,8]
n = 20

The `note_list` represents the music pieace we would like to harmonize.

In [120]:
note_list = [1, 4, 3, 5, 1, 7, 8, 3, 5, 4, 3, 5, 4, 3, 6, 5, 4, 3, 5, 1]

We would like to generate a triad. Hence, as opposed to single note constraint, we have the constraint forcing that there are three notes at each time point $
\sum_{j \in P} x_{i,j} = 3
$. (Eq.41)



In [121]:
H = 0
for i in range(1,n+1):
    H += 2*Constraint((3-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Triad_{i}")

We would like to have the first and the last triad to be built on the first degree of the scale, hence we use the following constraints. (Eq. 42)

In [122]:
H += Constraint((1-Binary(f"x_{1}_{3}"))**2, f"note_{1}_3")
H += Constraint((1-Binary(f"x_{1}_{5}"))**2, f"note_{1}_5")
H += Constraint((1-Binary(f"x_{20}_{3}"))**2, f"note_{20}_3")
H += Constraint((1-Binary(f"x_{20}_{5}"))**2, f"note_{20}_5")
H += Constraint((1-Binary(f"x_{20}_{1}"))**2, f"note_{20}_1")

Each triad should contain the note given in the `note_list`.

In [123]:
for i in range(1,n+1):
    H += Constraint((1-Binary(f"x_{i}_{note_list[(i-1)]}"))**2, f"note_{i}")

We would like to avoid consecutive degrees to appear in the triad, as well as degrees 1-7, 2-6, 1-8. Hence we add the following penalty terms to the objective. (Eq. 44)

In [124]:
A = [1,6,7]
for i in range(1,n+1):
    for j, k in product(P,P):
        if abs(j-k) in A:
            H += Binary(f"x_{i}_{j}")*(Binary(f"x_{i}_{k}"))

<b> Running the experiment

In [125]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

In [126]:
sampleset = anneal(bqm, 'h')

<b> Interpreting the results

We use the method `translate_chord` to convert our sample into a list of chords.

In [127]:
for data in sampleset.data():
    seq = translate_chord(data.sample,P,n)
    print(seq,data.energy)

{1: [1, 3, 5], 2: [4, 6, 8], 3: [3, 5, 7], 4: [3, 5, 7], 5: [1, 3, 5], 6: [3, 5, 7], 7: [3, 5, 8], 8: [3, 6, 8], 9: [3, 5, 7], 10: [2, 4, 7], 11: [3, 6, 8], 12: [2, 5, 7], 13: [1, 4, 6], 14: [3, 5, 7], 15: [1, 4, 6], 16: [1, 3, 5], 17: [2, 4, 7], 18: [3, 5, 8], 19: [1, 3, 5], 20: [1, 3, 5]} 0.0


You can check if any of the constraints are broken or not.

In [128]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

{}


In [129]:
filename = f"experiment8"
store_results(filename, sampleset)